# Pacman A* Search

*by Kira Miller and Amber Nolte, December 8th, 2018*

This journal inlcudes an analysis of the history of the A\* Search Algorithm, the current uses of it, and the predicted future of the algorithm. It also includes an initial implementation of the Pacman game using the A\* Search alogrithm. 

## GUI Implementation

In order to be able to see the results and progress of the Pacman game, a GUI was necessary to implement. Since no team members have experience with GUI implementations, we looked into already implemented GUIs that would be able to be incorporated into our project. The majority of Pacman GUI libraries were complicated and included lots of additional features that we did not need for this project. We ended up stumbling upon a Pacman GUI that was implemented in java using the Processing application. Processing is a software sketchbook that allows you to easily convert code into visual representations. The initial Pacman Processing implementation was a Pacman maze with 4 ghosts, and Pacman. Dots were displayed on the maze so Pacman could gather points and big dots were displayed in order to allow Pacman to eat the ghosts. We adjusted the GUI in order to change it to a Christmas themed Pacman game. Pacman became the Grinch by replacing the original Pacman circle with an image of the Grinch. Pacman/Grinch was still user controlled with the arrow keys. For each ghost, a similar process was executed where we replaced the various ghosts with different holiday characters (a reindeer, a snowman, an elf, and Santa). In addition to this, we implemented a counter that would cause the dots to blink between red and green colors every second. The maze was also adjusted by adding holiday decorations such as snowflakes and Christmas lights. When the game begins, a .wav file is uploaded into the project and plays Jingle Bells to acompany the Christmas GUI theme. All these changes involved learning the Processing functions, libraries, and various formats.

Below is a video of the Pacman game GUI that we had altered and successfully implemented. The functionality is due to the project source that we used as a base to build our GUI off of. 

In [19]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/ny_8CkKDyds" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

After successfully implementing the GUI, we began to look at how to incorporate the GUI code with the A\* search implementation. The project that we used in order to get a base GUI working used vectors, paths, nodes, and other classes that implemented the functionality of Pacman. The structure we planned for our backend implementation included a Pacman class, ghost class, maze class, and run class. This target structure was extremely different than the already implemented version in Processing, and in order to incorporate them, the GUI would have to be rewritten which would be a big challenge given that neither of us have extensive experience with front end developement. Because of the extreme amount of changes we would have to make to the Processing GUI code, we ultimately decided to abandon the Processing implementation. Because of this, we left the GUI work that has been done so far in the past in order to focus on the A\* portion of the project. This decision allowed us to implement the Pacman A\* search with the structure we wanted without worrying about the effects our changes would have on the GUI.

## A* Search Implementation

The Pacman implementation consists of four classes: Pacman.py, Ghost.py, Maze.py, Play.py. Initially in the project proposal we explored the idea of multithreading the project so each instance of Ghost.py would be a thread which would all represent various ghosts. However, after looking into it and planning the project structure, we decided to have one ghost. The ghost is what will be using A* search in order to find the most efficient path to Pacman's current location. In our implementation, Pacman is not user controlled and will focus on eating dots regardless of the location of the ghost. Pacman searches for the nearest dot and pursues it in order to get the most points possible.

Below is the implementation of the Play.py class which initiates the game and creates instances of the maze, Pacman, and the ghost. The majority of the code came from the A3 assignment including the Node class, aStarSearch method, the aStarSearchHelper method, and the runHelperAStar method. This implementation varies from A3 largely since A3 included start and end states, but in Pacman the start and end states are dynamic; the states are locations. The start state is the spawn location of the ghost, while the end state is not a state but a condition where the ghost location is equal to the Pacman location.

In [33]:
import time

class Node:
    def __init__(self, state, f=0, g=0, h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h

    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    h = hF(startState)
    startNode = Node(state=startState, f=0 + h, g=0, h=h)
    return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))

def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    if goalTestF(parentNode.state):
        return ([parentNode.state], parentNode.g)
    ## Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    if not actions:
        return ("failure", float('inf'))
    children = []
    for action in actions:
        (childState, stepCost) = takeActionF(parentNode.state, action)
        global gNumNodes
        gNumNodes += 1
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h + g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
    while True:
        # find best child
        children.sort(key=lambda n: n.f)  # sort by f value
        bestChild = children[0]
        if bestChild.f > fmax:
            return ("failure", bestChild.f)
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result, bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF,
                                                hF, min(fmax, alternativef))
        if result is not "failure":
            result.insert(0, parentNode.state)
            return (result, bestChild.f)

def runHelperAStar(startState, goalState, heuristic):
    startTime = time.time()
    global gNumNodes
    gNumNodes = 0
    if heuristic is 1:
        h1Path, h1Depth = aStarSearch(startState, Santa.actions, Santa.takeAction, lambda s: Santa.goalTest(s),
                                      lambda s: h1(s, goalState))
        endTime = time.time()
        return h1Depth, gNumNodes, endTime - startTime
    elif heuristic is 2:
        h2Path, h2Depth = aStarSearch(startState, Santa.actions, Santa.takeAction, lambda s: Santa.goalTest(s),
                                      lambda s: h2(s, goalState))
        endTime = time.time()
        return h2Depth, gNumNodes, endTime - startTime
    else:
        h3Path, h3Depth = aStarSearch(startState, Santa.actions, Santa.takeAction, lambda s: Santa.goalTest(s),
                                      lambda s: h3(s, goalState))
        endTime = time.time()
        return h3Depth, gNumNodes, endTime - startTime

def h1(state, goal):
    return 0

def h2(state, goal):
    return 1

def h3(state, goal):
    return manhattanDistance(state, goal)

def manhattanDistance(state, goal):
    return abs(state[0] - goal[0]) + abs(state[1] - goal[1])

if __name__ == "__main__":
    board = [['X', 'X', 'X', 'X', 'X', 'X', 'X'],
             ['X', ' ', '.', '.', '.', 'G', 'X'],
             ['X', '.', 'X', 'X', 'X', '.', 'X'],
             ['X', ' ', ' ', '.', ' ', ' ', 'X'],
             ['X', '.', 'X', 'X', 'X', '.', 'X'],
             ['X', 'S', '.', '.', '.', ' ', 'X'],
             ['X', 'X', 'X', 'X', 'X', 'X', 'X']]

Below is the Dot and Maze classes. The Dot class just holds the coordinates and location of each instance of the class (each dot on the maze has its own dot class instance). The Maze class keeps track of the state of the maze. It has 7 class variables: maze keeps track of the current 'state' of the maze, ghost is the location of the ghost, pacman is the location of pacman, dotlist keeps track of the dots on the map and whether they have been eaten by pacman or not, height and length are the measurements of the maze, and remaining dots counts the number of dots in the maze. When makeMove is called by Ghost.py or Pacman.py, it updates the maze for the game. ghostMove() allows the ghost to move around the maze without eating dots. When a ghost is 'standing' on a dot, the ghost 'G' will be shown. Then, when the ghost moves from that spot the '.' is regenerated since dots cannot be eaten by ghosts. pacmanMove() updates the dotList as well as decrements the number of dots in the maze as pacman moves around and eats the dots.

In [31]:
import copy

class Dot:
    def __init__(self, x, y):
        self.location = x, y

    def location(self):
        return self.location

class Maze:
    def __init__(self, ogMaze):
        self.maze = copy.deepcopy(ogMaze)
        self.ghost = (1,5)
        self.pacman = (5,1)
        self.dotList = self.createDotList(ogMaze)
        self.height = 7
        self.length = 7
        self.remainingDots = 11

    # even if the ghost moves over the dot, it needs to stay there
    def createDotList(self, maze):
        allDots = []
        for x, a in enumerate(maze):
            for y, b in enumerate(maze[x]):
                if b is '.':
                    allDots.append(Dot(x, y))
                if b is ' ':
                    allDots.append(Dot(x, y))
        return allDots

    def makeMove(self, past, x, y):
        if isinstance(past, Santa.Ghost):
            Maze.ghostMove(self, past, x, y)
        if isinstance(past, Grinch.Pacman):
            Maze.pacmanMove(self, past, x, y)

    def ghostMove(self, past, futureX, futureY):
        pastX = past.location[0]
        pastY = past.location[1]
        # if the ghost is going over a dot, it's stored so that the
        # dot can be restored after the ghost moves out of that spot
        self.maze[pastX][pastY] = 'S'
        if past.coveringDot:
            self.maze[pastX][pastY] = '.'
            past.onDot = False
        elif self.maze[futureX][futureY] is '.':
            self.maze[pastX][pastY] = ' '
            past.coveringDot = True
            past.location = pastX, pastY
        else:
            self.maze[pastX][pastY] = ' '
        past.location = pastX, pastY

    def pacmanMove(self, past, futureX, futureY):
        pastX = past.location[0]
        pastY = past.location[1]
        self.maze[pastX][pastY] = 'G'
        self.maze[pastX][pastY] = ' '
        past.location = futureX, futureY
        # if he ate a dot
        if self.maze[pastX][pastY] is '.':
            self.remainingDots -= self.remainingDots
            for dot in self.dotList:
                if dot.location[0] == pastX and dot.location[1] == pastY:
                    self.dotList.remove(dot)

The Ghost.py class below implements the helper functions that used by the A\* search algorithm. Ghost.py has 3 class variables: location which keeps an updated tuple of the coordinates of the ghost, a boolean coveringDot, and an instance of the current state of the maze. A simple getter for the location variable is implemented so it can be accessed by Play.py in order to see what the current location of the ghost is. In addition to this, actions() has a list of walls since the ghost cannot run into walls, and anything that is not a wall (right, left, up, down) is returned as a list of possible actions. takeAction() calls the makeMove() method in maze that updates the maze with the new position of the ghost. goalTest() compares if the goal has been reached, and since there is no fixed goal state, it simply checks to see if the ghost location is the same as the pacman location, and if so the ghost has caught pacman and the game is over. The last function getPacmanLocation() is a helper method used by goalTest() to search the current state of the maze for pacman's most recent location.

In [35]:
class Ghost:
    def __init__(self, Maze):
        self.location = (1,5)
        self.coveringDot = False
        self.maze = Maze

    def getLocation(self):
        return self.location

    def actions(self):
        walls = [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6),
                 (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0),
                 (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6),
                 (1, 6), (2, 6), (3, 6), (4, 6), (5, 6),
                 (2, 2), (3, 2), (4, 2),
                 (2, 4), (3, 4), (4, 4)]
        validActions = []
        currentLocation = self.location
        right = (currentLocation[0] + 1, currentLocation[1])
        left = (currentLocation[0] - 1, currentLocation[1])
        up = (currentLocation[0], currentLocation[1] + 1)
        down = (currentLocation[0], currentLocation[1] - 1)
        if right not in walls:
            validActions.append("right")
        if left not in walls:
            validActions.append("left")
        if up not in walls:
            validActions.append("up")
        if down not in walls:
            validActions.append("down")
        return validActions

    def takeAction(self, action):
        tLocation = self.location
        newLocation = ()
        if action == "right":
            newLocation = (tLocation[0]+1, tLocation[1])
        elif action == "left":
            newLocation = (tLocation[0]-1, tLocation[1])
        elif action == "up":
            newLocation = (tLocation[0], tLocation[1]+1)
        elif action == "down":
            newLocation = (tLocation[0], tLocation[1]-1)
        self.maze.makeMove(self, newLocation[0], newLocation[1])
        self.location = newLocation

    def goalTest(self):
        if Ghost.getPacmanLocation(self) == self.location:
            return True
        return False

    def getPacmanLocation(self):
        myMaze = self.maze.maze
        pacmanCoordinates = ()
        for r in range(len(myMaze)):
            for el in range(len(myMaze[0])):
                if myMaze[r][el] == "G":
                    pacmanCoordinates = (r, el)
        return pacmanCoordinates


The Pacman.py class is implemented below and has 2 class variables: location which keeps an updated location of Pacman, as well an an instance of the current state of the maze. Like Ghost.py, there is a getLocation() to allow Play.py to quickly get the most recent location of Pacman. actions() has the same implementation as ghost by allowing all actions that do not run Pacman into a wall. takeAction() is similar to the Ghost.py implementation except that it will call makeMove() and therefore pacmanMove() in maze which will allow Pacman to eat dots and remove the eaten dots from the maze. The takeShortest() method finds the closest dot to Pacman's current location which tells Pacman which direction he should move in. It first gets all the possible actions based off of pacman's current location,  then for each of those possible actions it calculates the distance from that action to the nearest dot. It does this comparison in testAction() where the tested action is taken and the the distance from that new location to the nearest dot is found. testAction() returns the smallest distance (the distance to the closest dot after the possible action is tested) for the given action. After getting the smallest distance from a dot for any of the actions, the action picked depends on which action led to the shortest path to the nearest dot.

In [34]:
class Pacman:
    def __init__(self, Maze):
        self.location = (5,1)
        self.maze = Maze

    def getLocation(self):
        return self.location

    def actions(self):
        walls = [(0,0),(0,1),(0,2),(0,3),(0,4),(0,5),(0,6),
                 (1,0),(2,0),(3,0),(4,0),(5,0),(6,0),
                 (6,1),(6,2),(6,3),(6,4),(6,5),(6,6),
                 (1,6),(2,6),(3,6),(4,6),(5,6),
                 (2,2),(3,2),(4,2),
                 (2,4),(3,4),(4,4)]
        validActions = []
        currentLocation = self.location
        right = (currentLocation[0] + 1, currentLocation[1])
        left = (currentLocation[0] - 1, currentLocation[1])
        up = (currentLocation[0], currentLocation[1] + 1)
        down = (currentLocation[0], currentLocation[1] - 1)
        if right not in walls:
            validActions.append("right")
        if left not in walls:
            validActions.append("left")
        if up not in walls:
            validActions.append("up")
        if down not in walls:
            validActions.append("down")
        return validActions

    def takeAction(self, action):
        tLocation = self.location
        newLocation = ()
        if action == "right":
            newLocation = (tLocation[0] + 1, tLocation[1])
        elif action == "left":
            newLocation = (tLocation[0] - 1, tLocation[1])
        elif action == "up":
            newLocation = (tLocation[0], tLocation[1] + 1)
        elif action == "down":
            newLocation = (tLocation[0], tLocation[1] - 1)
        self.maze.makeMove(self, newLocation[0], newLocation[1])
        self.location = newLocation

    def takeShortest(self, maze):
        possibleActions = Pacman.actions(self)
        shortestDistance = 100
        bestAction = ""
        for a in possibleActions:
            tempDistance = Pacman.testAction(self, a, maze) #distance to closest dot
            if shortestDistance > tempDistance:
                shortestDistance = tempDistance
                bestAction = a
        Pacman.takeAction(self, bestAction)

    def testAction(self, action, maze): #specified action, which dot is closest
        tempLocation = self.location
        if action == "right":
            newX = tempLocation[0] + 1
            tempLocation = (newX, tempLocation[1])
        elif action == "left":
            newX = tempLocation[0] - 1
            tempLocation = (newX, tempLocation[1])
        elif action == "up":
            newY = tempLocation[1] + 1
            tempLocation = (tempLocation[0], newY)
        elif action == "down":
            newY = tempLocation[1] - 1
            tempLocation = (tempLocation[0], newY)
        allDots = Pacman.getDots(maze)
        minDistance = 100
        for d in allDots:
            testDistance = Pacman.distance(self, tempLocation, d)
            if testDistance < minDistance:
                minDistance = testDistance
        return minDistance

    def getDots(self):
        myMaze = self.maze.maze
        dotLocations = []
        for r in range(len(myMaze)):
            for el in range(len(myMaze[0])):
                if myMaze[r][el] == '.':
                    dotLocations.append((r, el))
        return dotLocations

    def distance(self, l1, l2):
        return abs(l1[0] - l2[0]) + abs(l1[1] - l2[1])

## Methods

The main resource used was for the initial GUI demonstrated in the YouTube link above. The link to the github we used for that GUI portion is https://github.com/Code-Bullet/PacmanGame/blob/master/PacmanGame/PacmanGame.pde

Although we ended up not using the Processing application or the GUI from it, we learned a lot about the Processing app and how frontend programming connects, communicates, and interacts with backend processes.

Kira Miller wrote the implementations for Play.py and Maze.py. In addition to this, Kira also did the research below for A\* search and current reasearch and applications for this AI algorithm.

Amber Nolte implemented the Christmas Pacman GUI using the Processing application. In addition to this, she wrote the Ghost.py and Pacman.py classes as well as the summary of every class and the summary of the original GUI process and implementation. The methods, results, and conclusion was also written by Amber.

## Results

The results of this study were inconclusive as we were not able to get the full functionality of the A\* search Pacman program. The various heuristic functions would change the outcome of the algorithm, although analyzing this change would be difficult. In the original GUI impementation, Pacman was user-controlled using the arrow keys. Because of this, changing the heuristic functions may not cause drastic or even accurate changes to the results since having a user controlled portion of the game causes too many variables. For example, as the uesr plays the game repetitively with each heuristic, they become better at the game and come up with strategies to avoid the ghosts. In order to make the project more objective and leave less to human error, Pacman would ideally not be user controlled and also use the A\* search algorithm. Instead of searching for something like the ghosts are, Pacman would be avoiding the ghosts and perhaps searching for dots as well. This would also need to incorporate a priority hierarchy if it comes down to getting a dot versus going towards a ghost. With big dots that allow Pacman to eat ghosts, he would potentially pick the big dot over running into a ghost, but choose to run away from a ghost if only a small dot must be sacrificed.

## Conclusions

From this project, we gained experience with front end programming and how the front end GUI connects to the back end implementation. In addition to this, we learned how to apply A\* search to locations instead of states. This change from A3 taught us to compare dynamic goal states instead of constant states. The object oriented structure challenged our python coding experience as our heavily object oriented projects in the past have been implemented with java. A few different portions of this project were challenging such as the GUI, planning the structure of the implementation, and combining both the GUI and the implementation. The GUI as stated before was a challenge given that neither of us had experience with front end programming, but altering an existing GUI helped make the transition easier. Once we decided how we wanted to structure our classes, we realized the GUI that had been implemented would be difficult to merge with the new A\* search implementation. Due to this, we unfortunately could not use the GUI that Amber had been assigned to work on and finished, and we had to start from scratch. The backend portion was then written in python since we no longer needed to use java to support the GUI. Due to this, the timeline had to change. The GUI was finished but unfortunately the backend code had not been cosidered or planned so it caused a clash between the GUI and the A\* search implementation. The challenge of having multiple classes and moving parts between them caused some issues with the Play.py class, so the game does is not fully implemented. The greatest challenge of this project was all the parts of the maze that needed to be updated as the game progressed. The ghost and Pacman classes both need to have up-to-date instances of the maze, as well as the maze having the current location of the ghost and Pacman throughout the game.

## Research

### References

* [Goodfellow, et al., 2016] Ian Goodfellow and Yoshua Bengio and Aaron Courville, [Deep Learning](http://www.deeplearningbook.org), MIT Press. 2014.

Your report for a single person team should contain approximately 2,000 to 4,000 words, in markdown cells.  You can count words by running the following python code in your report directory.  Projects with two people should contain 4,000 to 8,000 words in their reports. Three-person teams should produce reports with 6,000 to 12,000 words, and so on.

Do not forget to include the following code cell at the bottom of your report.

In [7]:
import io
from IPython.nbformat import current
import glob
nbfile = glob.glob('Pacman A* Search Algorithm .ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Pacman A* Search Algorithm .ipynb is 328


When you check-in your project, you may tar or zip your files together and submit the .tar or .zip file.